In [1]:
import typing

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
import config
import src
import requests
import tqdm
import json
import numpy as np
import logging
import torch
device = torch.device('cpu')
from transformers import AutoTokenizer, AutoModel
#import cltrier_lib as lib

import yaml

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sstolwijk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/sstolwijk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/sstolwijk/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CFG = config.Config()

In [47]:
pd.set_option('display.max_colwidth', 100) 

In [3]:
#load data
reddit: pd.DataFrame = pd.read_parquet('data/Coarse_discourse_Reddit/redditdata.parquet') 
reddit

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author
0,t3_1bx6qw,[removed],100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None
1,t1_c9b2nyd,I've wanted to watch this for a long time. I w...,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10
2,t1_c9b30i1,You strike me as the type who would appreciate...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10
4,t3_omv7p,"Alright guys, little background about myself. ...",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus
...,...,...,...,...,...,...,...,...,...
116352,t1_d0mhap4,I didn't notice it until it was too late. What...,zootopia,Frozen 2,t3_48toyo,answer,None,1.0,Portgas
116353,t1_d0n2wdx,"I don't know, it was to quick.\nI just liked t...",zootopia,Frozen 2,t1_d0mhap4,answer,None,2.0,Pradfanne
116354,t1_d0msypp,The Frozen parody was on the right edge of the...,zootopia,Frozen 2,t1_d0mhap4,answer,None,2.0,SaucyServine
116355,t1_d0n4bpa,Much rather see more zootopia.,zootopia,Frozen 2,t3_48toyo,None,None,1.0,Jackofdemons


In [19]:
#clean empty body, when body is [removed] or [deleted]:
reddit['body'] = reddit['body'].replace('[deleted]', '').replace('[removed]', '')

In [5]:
#change options to low temperature (0,1) and compare result:
options_low = """
seed: 42
temperature: 0.1
"""

options_low = yaml.safe_load(options_low)

In [6]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

In [7]:
MODEL: str = 'llama3.1:70b-instruct-q6_K'  # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [20]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                        + f'Check whether your answer only consists of a list of claims. \n"Posts":\n<{reddit["body"][2]}>',
                                'options': options_low
                                }).json()

{'id': '9b17444a-816a-11ef-8a91-0d8e6fdd3d4b',
 'timestamp': '2024-10-03T09:33:54.028673',
 'model': 'llama3.1:70b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\n\n        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.\n        Claims are defined as the main assertion or conclusion of an argument.\n        You summarize each claim into a short simple sentence.\n\n        Response format:\n\n        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].\n\n        Response format template:\n        \n        ["claim 1", "claim 2", ... "claim x"]\n\t'},
  {'role': 'user',
   'content': 'The following set of social media posts are replies

In [36]:
#note these reddit comments are not necessarily replies to news or infotainment... perhaps adjust the context?
#also they are parsed sequentially not in batches/collections

In [21]:
reddit3000=reddit[:3000]

In [22]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(reddit3000.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["body"]}>',
                            'options': options_low
                            }).json()['response']                       
                ],
                columns=['claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

3000it [1:45:01,  2.10s/it]


In [23]:
chunked_result[:5]

[  claims
 0     [],
                                                                                                                  claims
 0  ["The user has wanted to watch something for a long time", "The country western aspect of something is off-putting"],
                                                                                                                                   claims
 0  ["Magnolia has a similar structure to Altman's films with no lead character", "Boogie Nights and Magnolia were influenced by Altman"],
                                                                                   claims
 0  ["Altman is famous for his ensemble casts", "The user has never seen an Altman film"],
                                                                                                                                                                                          claims
 0  ["The writer has had modest success with women in the past", "The writer lacks ki

In [24]:
claim_low = pd.concat(chunked_result, ignore_index=True)
print(claim_low)

                                                                                                                                                                                            claims
0                                                                                                                                                                                               []
1                                                                             ["The user has wanted to watch something for a long time", "The country western aspect of something is off-putting"]
2                                                            ["Magnolia has a similar structure to Altman's films with no lead character", "Boogie Nights and Magnolia were influenced by Altman"]
3                                                                                                            ["Altman is famous for his ensemble casts", "The user has never seen an Altman film"]
4     ["The writer has ha

In [25]:
claim_low.head()

,claims
0,[]
1,"[""The user has wanted to watch something for a long time"", ""The country western aspect of something is off-putting""]"
2,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and Magnolia were influenced by Altman""]"
3,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]"
4,"[""The writer has had modest success with women in the past"", ""The writer lacks killer instinct and extreme alpha attitude"", ""The writer's goal is to approach 100 sets by the end of April""]"


In [26]:
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10
2,t1_c9b30i1,"You strike me as the type who would appreciate it. I would give it a go. This is also my first Altman film so I didn't really know what to expect, except that people always compare PTA's Boogie Nights and Magnolia as being influenced by Altman. Magnolia is probably the best analog in terms of structure (having no lead character) but it is stylistically very different, much more melodramatic and transparently earnest.",100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never seen an Altman.",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10
4,t3_omv7p,"Alright guys, little background about myself. I'm a good looking, 23 year old male, have had modest success in the past with women, but have decided that modest just isn't good enough anymore.\n\nThe problem has never been a lack of attention, or opportunity. It's just been not having the killer instinct and extreme AA, to the point where women will basically EYE FUCK me, and I still don't have the testicular fortitude to say anything unless they say something first.\n\nMy goal is to approach 100 sets by the end of April, and hopefully break my AA.",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus


In [27]:
reddit3000.loc[:, "claim_oplow"] = claim_low

/tmp/ipykernel_1255/2298628496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit3000.loc[:, "claim_oplow"]=claim_low


In [28]:
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[]
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a long time"", ""The country western aspect of something is off-putting""]"
2,t1_c9b30i1,"You strike me as the type who would appreciate it. I would give it a go. This is also my first Altman film so I didn't really know what to expect, except that people always compare PTA's Boogie Nights and Magnolia as being influenced by Altman. Magnolia is probably the best analog in terms of structure (having no lead character) but it is stylistically very different, much more melodramatic and transparently earnest.",100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and Magnolia were influenced by Altman""]"
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never seen an Altman.",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]"
4,t3_omv7p,"Alright guys, little background about myself. I'm a good looking, 23 year old male, have had modest success in the past with women, but have decided that modest just isn't good enough anymore.\n\nThe problem has never been a lack of attention, or opportunity. It's just been not having the killer instinct and extreme AA, to the point where women will basically EYE FUCK me, and I still don't have the testicular fortitude to say anything unless they say something first.\n\nMy goal is to approach 100 sets by the end of April, and hopefully break my AA.",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus,"[""The writer has had modest success with women in the past"", ""The writer lacks killer instinct and extreme alpha attitude"", ""The writer's goal is to approach 100 sets by the end of April""]"


In [29]:
reddit3000.claim_oplow.isna().sum()

0

In [ ]:
#get MXBAI embeddings for the claims:
context = 'claims made in social media replies to a news- or infotainment-post'
claim_optlow_MXBAI: typing.Dict[str, np.ndarray] = (
    pd.Series(src.SentenceEmbedder()(reddit3000["claim_oplow"], prefix= 'You help me get embeddings for a sentence. I provide you with a context and a sentence and you reply only with that exact sentence. Context = ' + context + '; Sentence: '), name="claim_optlow_MXBAI")
    .rename_axis(index=reddit3000.index.names)
)
claim_optlow_MXBAI.head()

In [ ]:
reddit3000 = reddit3000.join(claim_optlow_MXBAI)
reddit3000.to_parquet(f'{CFG.report_dir}/reddit3000.parquet')
reddit3000.head()


In [32]:
print(reddit3000['thread_title'].value_counts().head(30))

thread_title
If your country has a 30-35 hour work week, what do you do with your free time?                                                                                           41
Get rid of unwanted system apps (adb shell pm hide)                                                                                                                       41
What ROM are you currently running and what do you like/dislike about it?                                                                                                 41
I'm ChrisAcrosstheWorld, Japan's first salaried foreign NINJA -- AMA                                                                                                      39
Let's post starting salary offers for new hires of this year in audit, tax, advisory.                                                                                     38
Hey AskEngineers, can you give me some hope - what cool things have you done at your job?                                 

In [33]:
#select the rows in the dataset that have the a showName with more than cutoff comments:   
cutoff = 35
selectedposts = reddit3000[reddit3000['thread_title'].isin(reddit3000['thread_title'].value_counts()[reddit3000['thread_title'].value_counts() >= cutoff].index)]

In [48]:
selectedposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
600,t3_1hrkr9,I have recently noticed that music only works for me if it is very loud. If I go half volume I l...,adhd,Trying to understand why loud music helps with ADHD for me.,none,question,True,NaN,MrDurp,"[""The writer only enjoys music when it's very loud"", ""The writer loses interest in music quickly...","[0.15566717088222504, -0.12680421769618988, -0.5164549946784973, 0.4246806204319, -0.00983746349..."
601,t1_cax7ehx,"I don't listen to music very loud, but I do listen to it loud enough that it cancels out noise a...",adhd,Trying to understand why loud music helps with ADHD for me.,t3_1hrkr9,None,None,1.0,ChrisBacon,"[""The writer can concentrate and study with music"", ""The writer cannot concentrate with people t...","[0.2486271858215332, -0.20359307527542114, -0.4116734564304352, 0.3750465214252472, 0.1924118697..."
602,t1_caxg8mm,"Is it possibly because the music is predictable, whereas people talking is novel? If hear a new ...",adhd,Trying to understand why loud music helps with ADHD for me.,t1_cax7ehx,None,None,2.0,Sadiew1990,"[""Predictable music can be less distracting than unpredictable music"", ""Familiar songs can stimu...","[0.20179954171180725, -0.0002015717327594757, -0.510717511177063, 0.2707652449607849, -0.0343725..."
603,t1_caxwdza,"I think this is pretty spot on, as it's something I've contemplated many times.",adhd,Trying to understand why loud music helps with ADHD for me.,t1_caxg8mm,None,None,3.0,ChrisBacon,"[""This analysis is accurate""]","[0.3507775664329529, 0.07732725888490677, -0.5983396172523499, 0.24801413714885712, 0.3272857666..."
604,t1_cay1aso,"Huh. I have the opposite reaction. If I'm writing an essay, and I'm listening to a song I don't ...",adhd,Trying to understand why loud music helps with ADHD for me.,t1_caxg8mm,None,None,3.0,None,"[""The writer phases out unknown songs while writing an essay"", ""The writer hears lyrics of known...","[0.4403245747089386, -0.1945212483406067, -0.3513432443141937, 0.2210473269224167, 0.22260612249..."


In [35]:
selectedposts.claim_optlow_MXBAI.isna().sum()

0

In [84]:
selectedposts = selectedposts.dropna(subset=['claim_optlow_MXBAI'])

In [37]:
grouped_data = selectedposts.groupby("thread_title")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['claim_optlow_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['claim_optlow_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

 10%|█         | 1/10 [00:00<00:01,  5.78it/s]

Get rid of unwanted system apps (adb shell pm hide):Get rid of unwanted system apps (adb shell pm hide):4.73830592394042
Get rid of unwanted system apps (adb shell pm hide):Guys, how do you deal with the dreaded double-stream?:6.380127243347456
Get rid of unwanted system apps (adb shell pm hide):Hey AskEngineers, can you give me some hope - what cool things have you done at your job?:6.286547061337251
Get rid of unwanted system apps (adb shell pm hide):I'm ChrisAcrosstheWorld, Japan's first salaried foreign NINJA -- AMA:4.479606852501545
Get rid of unwanted system apps (adb shell pm hide):If your country has a 30-35 hour work week, what do you do with your free time?:6.280382520802147
Get rid of unwanted system apps (adb shell pm hide):If your roommate was constantly negative and complaining, and you had already tried to make them stop but it didn't work, what would you do?:5.717101281616605
Get rid of unwanted system apps (adb shell pm hide):Let's post starting salary offers for new h


  0%|          | 0/10 [00:00<?, ?it/s]

Guys, how do you deal with the dreaded double-stream?:Guys, how do you deal with the dreaded double-stream?:5.819947235546318
Guys, how do you deal with the dreaded double-stream?:Hey AskEngineers, can you give me some hope - what cool things have you done at your job?:6.9050352300985995
Guys, how do you deal with the dreaded double-stream?:I'm ChrisAcrosstheWorld, Japan's first salaried foreign NINJA -- AMA:5.626086604246866
Guys, how do you deal with the dreaded double-stream?:If your country has a 30-35 hour work week, what do you do with your free time?:6.8393780022048425
Guys, how do you deal with the dreaded double-stream?:If your roommate was constantly negative and complaining, and you had already tried to make them stop but it didn't work, what would you do?:6.400754932855487
Guys, how do you deal with the dreaded double-stream?:Let's post starting salary offers for new hires of this year in audit, tax, advisory.:6.9417086226391325
Guys, how do you deal with the dreaded double


 20%|██        | 2/10 [00:00<00:01,  6.90it/s]

...

What ROM are you currently running and what do you like/dislike about it?:What ROM are you currently running and what do you like/dislike about it?:5.412314982940632


 90%|█████████ | 9/10 [00:00<00:00, 15.42it/s]

What ROM are you currently running and what do you like/dislike about it?:What's the name of your LFS in your home town?:6.332520811622372



100%|██████████| 10/10 [00:00<00:00, 13.37it/s]

What's the name of your LFS in your home town?:What's the name of your LFS in your home town?:4.902105013902613


In [38]:
#select the rows in the dataset that have the a showName with LESS than cutoff comments for increased variance:   
cutoff = 5
fewselectposts = reddit3000[reddit3000['thread_title'].isin(reddit3000['thread_title'].value_counts()[reddit3000['thread_title'].value_counts() <= cutoff].index)]

In [39]:
fewselectposts.claim_optlow_MXBAI.isna().sum()

0

In [49]:
fewselectposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a long time"", ""The country western aspect of someth...","[0.4254729151725769, -0.01032189279794693, -0.6786631941795349, 0.32541653513908386, 0.298534691..."
2,t1_c9b30i1,You strike me as the type who would appreciate it. I would give it a go. This is also my first A...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and...","[0.7148110270500183, -0.15317122638225555, -0.5416110157966614, -0.1810915321111679, -0.36302334..."
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never se...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]","[0.7345885634422302, -0.177871972322464, -0.609611451625824, -0.0007055625319480896, -0.08616860..."
77,t3_25hor7,"To be honest, there aren't really too many ideas for a skill that are so in depth they could be ...",2007scape,New skill?,none,question,True,NaN,GrandmaTitz,"[""Jagex did a great job making the existing skills"", ""There aren't many ideas for a skill that c...","[0.5224913358688354, -0.2205544114112854, -0.7710671424865723, 0.10436008125543594, 0.1284627318..."


In [41]:
grouped_data = fewselectposts.groupby("thread_title")
dist = torch.nn.PairwiseDistance()
resultstopic: typing.Dict[typing.Tuple[str, str], float] = {}
for model_1, c_1 in tqdm.tqdm(grouped_data['claim_optlow_MXBAI'], total=grouped_data.ngroups):
    for model_2, c_2 in tqdm.tqdm(grouped_data['claim_optlow_MXBAI'], total=grouped_data.ngroups):

        if (
            (model_1, model_2) in resultstopic.keys() or 
            (model_2, model_1) in resultstopic.keys()
        ):
            continue

        res = sum([
            sum(dist(
                torch.tensor(np.array(v_1)), 
                torch.tensor(np.array(c_2.tolist()))
                )) / len(c_2)
            for v_1 in c_1
        ]) / len(c_1)

        resultstopic[(model_1, model_2)] = res

        print(f'{model_1}:{model_2}:{res.item()}')

100%|██████████| 66/66 [00:00<00:00, 2387.34it/s]


1-base Protoss Immortal Stalker/Sentry All-in:1-base Protoss Immortal Stalker/Sentry All-in:4.339136787327061
1-base Protoss Immortal Stalker/Sentry All-in:A possible solution to "nerf" TEAMING:8.589230770748259
1-base Protoss Immortal Stalker/Sentry All-in:A598454:9.684909007783705
1-base Protoss Immortal Stalker/Sentry All-in:Advice on Arduino Capabilities:9.78963743562481
1-base Protoss Immortal Stalker/Sentry All-in:Amd r9 380 driver crashing:10.104427390013619
1-base Protoss Immortal Stalker/Sentry All-in:Can I disolve gelatine again once cold?:9.64593702258983
1-base Protoss Immortal Stalker/Sentry All-in:Can I kill black demon at lv51?:8.898439491785478
1-base Protoss Immortal Stalker/Sentry All-in:Can i recover my corrupted USB?:9.718662791606878
1-base Protoss Immortal Stalker/Sentry All-in:Confession bear at khols:9.727718035951195
1-base Protoss Immortal Stalker/Sentry All-in:DTX120: #87 - Nashville:10.214850279114373
1-base Protoss Immortal Stalker/Sentry All-in:Dear sociol


100%|██████████| 66/66 [00:00<00:00, 2038.74it/s]


A possible solution to "nerf" TEAMING:A possible solution to "nerf" TEAMING:5.330079973625755
A possible solution to "nerf" TEAMING:A598454:8.707217159046538
A possible solution to "nerf" TEAMING:Advice on Arduino Capabilities:9.203460528399933
A possible solution to "nerf" TEAMING:Amd r9 380 driver crashing:9.148174192045595
A possible solution to "nerf" TEAMING:Can I disolve gelatine again once cold?:8.885723797658317
A possible solution to "nerf" TEAMING:Can I kill black demon at lv51?:8.489594677631494
A possible solution to "nerf" TEAMING:Can i recover my corrupted USB?:8.840144210718261
A possible solution to "nerf" TEAMING:Confession bear at khols:8.847193228975943
A possible solution to "nerf" TEAMING:DTX120: #87 - Nashville:9.428430946198299
A possible solution to "nerf" TEAMING:Dear sociologists of Reddit, what do you think of the social changes in a country such as Singapore, that has caused it to be stubborn towards government policies that try to address the declining birt


100%|██████████| 66/66 [00:00<00:00, 2476.18it/s]


...


100%|██████████| 66/66 [00:00<00:00, 32582.87it/s]


rex reclaim:rex reclaim:3.1999999999999965e-05
rex reclaim:rs authenticator:3.3178896673343474



100%|██████████| 66/66 [00:00<00:00, 36366.80it/s]

rs authenticator:rs authenticator:4.021004786027182



100%|██████████| 66/66 [00:01<00:00, 52.81it/s]


In [ ]:
#somewhat higher variance, but not much....

In [90]:
reddit3000.to_parquet(f'{CFG.report_dir}/reddit3000.parquet')

In [8]:
reddit3000= pd.read_parquet(f'{CFG.report_dir}/reddit3000.parquet') 

In [44]:
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a long time"", ""The country western aspect of someth...","[0.4254729151725769, -0.01032189279794693, -0.6786631941795349, 0.32541653513908386, 0.298534691..."
2,t1_c9b30i1,You strike me as the type who would appreciate it. I would give it a go. This is also my first A...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and...","[0.7148110270500183, -0.15317122638225555, -0.5416110157966614, -0.1810915321111679, -0.36302334..."
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never se...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]","[0.7345885634422302, -0.177871972322464, -0.609611451625824, -0.0007055625319480896, -0.08616860..."
4,t3_omv7p,"Alright guys, little background about myself. I'm a good looking, 23 year old male, have had mod...",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus,"[""The writer has had modest success with women in the past"", ""The writer lacks killer instinct a...","[0.5479655265808105, -0.197927325963974, -0.4277328550815582, 0.18708674609661102, 0.06689110398..."


In [45]:
pd.set_option('display.max_colwidth', None) 
reddit3000.loc[:3, ['body', 'claim_oplow']]

,body,claim_oplow
0,,[]
1,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,"[""The user has wanted to watch something for a long time"", ""The country western aspect of something is off-putting""]"
2,"You strike me as the type who would appreciate it. I would give it a go. This is also my first Altman film so I didn't really know what to expect, except that people always compare PTA's Boogie Nights and Magnolia as being influenced by Altman. Magnolia is probably the best analog in terms of structure (having no lead character) but it is stylistically very different, much more melodramatic and transparently earnest.","[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and Magnolia were influenced by Altman""]"
3,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never seen an Altman.","[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]"
